<a href="https://colab.research.google.com/github/ItsApex/Blockchain-Training-/blob/main/BeyondMerket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!ping www.google.com

/bin/bash: line 1: ping: command not found


In [ ]:
!pip show transformers

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:You are my scheduling Agent


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not your guy, scheduling agent.
>> User:I will give you a sentence analyse it and give me response in JSON


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not your guy, guy.
>> User:Ik you are not my Guy, Iam saying can you give me response in JSON


...

DialoGPT: Ik you are not my Guy, guy.
>> User:json


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Ik you are not my Guy, guy.
>> User:U r idiot


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Ik you are not my Guy, guy.


In [ ]:
import re
from datetime import datetime

def process_user_input(input_text):
    tasks = re.findall(r'(\d{1,2}:\d{1,2} [A-Za-z\s]+)', input_text)
    temp_dates = re.findall(r'(\d{2}/\d{2}/\d{4})', input_text)
    dates = [temp_dates[0]]

    if not temp_dates:
        now = datetime.now()
        dates.append(now.strftime("%d/%m/%Y"))

    times = re.findall(r'(\d{1,2}:\d{1,2})', input_text)

    if len(times) < len(tasks) * 2:
        times += ['09:00'] * ((len(tasks) * 2) - len(times))

    return tasks, dates, times

In [ ]:
def generate_json_response(tasks, dates, times):
    schedule = {}

    for i in range(0, len(tasks), 2):
        start_time = times[i]
        end_time = times[i + 1] if i + 1 < len(times) else (datetime.strptime(times[i], "%H:%M") + timedelta(hours=1)).strftime("%H:%M")

        date = dates[1] if len(dates) > 1 else dates[0]

        schedule[f"{start_time} - {end_time} {date}"] = tasks[i] + ' ' + tasks[i + 1]

    return json.dumps(schedule)

In [ ]:
def main():
    print("Enter a description of your tasks including the date, time, and their descriptions separated by a comma:")
    input_text = input("Your input: ")

    tasks, dates, times = process_user_input(input_text)

    json_response = generate_json_response(tasks, dates, times)

    print("JSON response:")
    print(json_response)

if __name__ == "__main__":
    main()

Enter a description of your tasks including the date, time, and their descriptions separated by a comma:
Your input: On 31/03/2024 at 8:00, walk 6km
JSON response:
{}


In [ ]:

def get_tasks():
    user_input = input("Enter your tasks with date, time, and description separated by a comma: ")
    return user_input

def parse_user_input(user_input):
    prompt = "If today's date is 12/2/2024 (its already in format of date/month/year ), understand the prompt  \"Today I will be walking from 11 to 12 and from 12 to 1 will be having my lunch and on 9 oc lciok I will take a bath\" then change the Date accordingly in format of Date/Month/Year. If day is not mentioned then consider it today else go with day in the prompt. Make this in the json format of {starttime-endtime-date : task,starttime-endtime-date: task...}. Make sure you only give me the json format in the mentioned manner strictly. If the endtime is not given then consider it would be 1 hour else consider the endtime. Don't write any text other than json. Make sure the time format is in 24 hours format and the date is in date/month/year format. "
    return prompt + " " + user_input

def extract_schedule(text):
    tasks = {}

    for line in text.split("\n"):
        if line:
            parts = re.findall(r"(\d{1,2}:\d{1,2})([- ]\d{1,2}:\d{1,2})? (.*)", line)
            if parts:
                start_time, end_time, description = parts[0]

                if not end_time:
                    end_time = (datetime.strptime(start_time, "%H:%M") + timedelta(hours=1)).strftime("%H:%M")

                date = datetime.now().strftime("%d/%m/%Y")

                if "to" in line:
                    start_time_obj = datetime.strptime(start_time, "%H:%M")
                    end_time_obj = datetime.strptime(end_time, "%H:%M")

                    if start_time_obj > end_time_obj:
                        end_date = datetime.now() + timedelta(days=1)
                        date = end_date.strftime("%d/%m/%Y")

                tasks[f"{start_time}-{end_time}-{date}"] = description

    return tasks

def main():
    user_input = get_tasks()
    parsed_input = parse_user_input(user_input)

    input_ids = tokenizer(parsed_input, return_tensors="pt").input_ids

    output = model.generate(
        input_ids,
        max_length=300,
        num_beams=4,
        early_stopping=True,
        return_dict_in_generate=True,
        output_scores=False,
        temperature=0.7,
    )

    generated_text = tokenizer.decode(output.sequences[0])

    schedule = extract_schedule(generated_text)

    print(json.dumps(schedule, indent=4))

if __name__ == "__main__":
    main()

Enter your tasks with date, time, and description separated by a comma: Today I will be walking from 11 to 12 and form 12 to 1 will be having my lunch and on 9 oc lciok I will take a bath


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{}


In [ ]:
import spacy
import json

# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_schedule(text):
    doc = nlp(text)
    schedule = {}
    current_activity = None
    for token in doc:
        if token.ent_type_ == "TIME":
            start_time = token.text
            end_time = None
            date = None
            for token in token.sent:
                if token.ent_type_ == "TIME" and token.text != start_time:
                    end_time = token.text
                if token.ent_type_ == "DATE":
                    date = token.text
            if not end_time:
                end_time = start_time
            if current_activity:
                schedule[f"{start_time}-{end_time} - {date}"] = current_activity
        elif token.ent_type_ == "EVENT":
            current_activity = token.text.capitalize()

    return schedule

def main():
    user_input = input("Please enter your schedule: ")
    schedule = extract_schedule(user_input)
    print(json.dumps(schedule, indent=4))

if __name__ == "__main__":
    main()


Please enter your schedule: On 31/03/2024 at 8:00 will walk, at 10:00 code, at 12:30 meet at 15:00 will have lunch
{}


In [ ]:
!pip install python-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#Restart runtime

In [ ]:
import spacy
import re
from datetime import datetime, timedelta

nlp = spacy.load('en_core_web_sm')

In [ ]:
def extract_date_time_task(user_input):
    doc = nlp(user_input)
    date = None
    time = None
    task = None

    for ent in doc.ents:
        if ent.label_ == 'DATE':
            date = datetime.strptime(ent.text, '%d/%m/%Y')
        elif ent.label_ == 'TIME':
            time = datetime.strptime(ent.text, '%H:%M')
        elif ent.label_ == 'EVENT':
            task = ent.text

    if not date:
        date = datetime.today()
    if not time:
        time = datetime.now().time()

    return date, time, task

In [ ]:
def format_time(t):
    return f"{t.hour:02d}:{t.minute:02d}"

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.4 MB/s eta 0:00:00
time: 417 µs (started: 2024-03-31 11:43:07 +00:00)


In [ ]:
import re
import datetime
import json

# Function to extract time, date, and task from user input
def extract_info(user_input):
    # Regular expressions to extract time, date, and task
    time_regex = r'\b(\d{1,2}(?::\d{2})?(?:\s*(?:am|pm))?)\b'
    date_regex = r'\b(\d{1,2}/\d{1,2}/\d{4})\b'
    task_regex = r'\b(.+)\b'

    # Extracting time
    time_matches = re.findall(time_regex, user_input)
    start_time = time_matches[0] if time_matches else ''
    end_time = time_matches[1] if len(time_matches) > 1 else start_time

    # Extracting date
    date_match = re.search(date_regex, user_input)
    date = date_match.group(1) if date_match else datetime.datetime.now().strftime('%d/%m/%Y')

    # Extracting task
    task_match = re.search(task_regex, user_input)
    task = task_match.group(1) if task_match else ''

    return start_time, end_time, date, task

def main():
    user_input = input("Please enter your schedule: ")

    # Extract information from user input
    start_time, end_time, date, task = extract_info(user_input)

    # Generate response in JSON format
    response = {f"{start_time}-{end_time} - {date}": task}

    # Output response
    print(json.dumps(response, indent=4))

if __name__ == "__main__":
    main()


Please enter your schedule: On 31/03/2003 at 8:00 I will walk at 12:00 will code and at 15:00 I will lunch
{
    "31-03 - 31/03/2003": "On 31/03/2003 at 8:00 I will walk at 12:00 will code and at 15:00 I will lunch"
}
time: 45.3 s (started: 2024-03-31 11:51:47 +00:00)


In [ ]:
import spacy
import json

# Load the spaCy English model with NER capabilities
nlp = spacy.load("en_core_web_sm")

# Function to extract date, time, and activity from user input using NER
def extract_info(user_input):
    doc = nlp(user_input)
    entities = {"DATE": None, "TIME": [], "ACTIVITY": None}

    for ent in doc.ents:
        if ent.label_ == "DATE":
            entities["DATE"] = ent.text
        elif ent.label_ == "TIME":
            entities["TIME"].append(ent.text)
        elif ent.label_ == "EVENT":
            entities["ACTIVITY"] = ent.text.strip()

    return entities

def main():
    user_input = input("Please enter your schedule: ")

    # Extract information from user input using NER
    extracted_info = extract_info(user_input)

    # Generate response in JSON format
    response = {}
    for time in extracted_info["TIME"]:
        response[f"{time}-{extracted_info['DATE']}"] = extracted_info["ACTIVITY"]

    # Output response
    print(json.dumps(response, indent=4))

if __name__ == "__main__":
    main()


Please enter your schedule: On 31/03/2003 at 8:00 I will walk at 12:00 will code and at 15:00 I will lunch
{
    "8:00-31/03/2003": null,
    "12:00-31/03/2003": null
}
time: 13.1 s (started: 2024-03-31 11:57:38 +00:00)


In [4]:
import pandas as pd

# List of activities
activities = [
    "meet", "walk", "run", "lunch", "dinner", "sports", "play", "soccer", "basketball",
    "tennis", "coding", "studying", "reading", "music", "listening", "gym", "exercise",
    "yoga", "meditation", "cooking", "baking", "gardening", "painting", "drawing",
    "writing", "blogging", "skiing", "snowboarding", "surfing", "swimming", "hiking",
    "camping", "fishing", "birdwatching", "photography", "volunteering", "travelling",
    "exploring", "learning", "teaching", "mentoring", "programming", "dancing",
    "singing", "acting", "watching movies", "playing games", "board games", "video games"
    # Add more activities as needed
]

# Create a DataFrame from the list of activities
activities_df = pd.DataFrame(activities, columns=["Activity"])

# Save the DataFrame to a CSV file
activities_df.to_csv("activities.csv", index=False)


In [5]:
import pandas as pd
import re
import spacy
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 24.5 s (started: 2024-03-31 12:29:48 +00:00)


In [6]:
activities_df = pd.read_csv("/content/activities.csv")

In [ ]:
# Display the loaded DataFrame
print(activities_df)

           Activity
0              meet
1              walk
2               run
3             lunch
4            dinner
5            sports
6              play
7            soccer
8        basketball
9            tennis
10           coding
11         studying
12          reading
13            music
14        listening
15              gym
16         exercise
17             yoga
18       meditation
19          cooking
20           baking
21        gardening
22         painting
23          drawing
24          writing
25         blogging
26           skiing
27     snowboarding
28          surfing
29         swimming
30           hiking
31          camping
32          fishing
33     birdwatching
34      photography
35     volunteering
36       travelling
37        exploring
38         learning
39         teaching
40        mentoring
41      programming
42          dancing
43          singing
44           acting
45  watching movies
46    playing games
47      board games
48      video games


In [10]:
import re
import spacy
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load the spaCy English model with NER capabilities
nlp = spacy.load("en_core_web_sm")

# Load the dataset
activities_df = pd.read_csv("/content/activities.csv")
activities_corpus = activities_df["Activity"].tolist()

# Function to train an activity classifier
def train_activity_classifier(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    y = [1] * len(corpus)
    clf = MultinomialNB()
    clf.fit(X, y)
    return vectorizer, clf

# Train
activity_vectorizer, activity_clf = train_activity_classifier(activities_corpus)

def print_features(vectorizer):
    print("Features extracted by the vectorizer:")
    feature_names = vectorizer.get_feature_names_out()
    print(feature_names)

# Print features extracted by the vectorizer
print_features(activity_vectorizer)

# Function to extract date, time, and activity using NER and activity classification
def extract_info(user_input):
    doc = nlp(user_input)
    entities = {"DATE": None, "TIME": [], "ACTIVITY": None}

    # time extraction
    time_regex = r'\b(\d{1,2}:\d{2}(?:\s*(?:am|pm))?)\b'
    time_matches = re.findall(time_regex, user_input)
    entities["TIME"] = time_matches

    for ent in doc.ents:
        if ent.label_ == "DATE":
            entities["DATE"] = ent.text
        elif ent.label_ == "EVENT":
            # Classify the event using the activity classifier
            X = activity_vectorizer.transform([ent.text])
            predicted_label = activity_clf.predict(X)
            if predicted_label[0] == 1:
                entities["ACTIVITY"] = ent.text.strip()

    print("Extracted information:", entities)
    return entities

def main():
    user_input = input("Please enter your schedule: ")

    # Extract information from user using NER and activity classification
    extracted_info = extract_info(user_input)

    # Generate JSON
    response = {}
    for time in extracted_info["TIME"]:
        response[f"{time}-{extracted_info['DATE']}"] = extracted_info["ACTIVITY"]

    # Output
    print(json.dumps(response, indent=4))

if __name__ == "__main__":
    main()


Features extracted by the vectorizer:
['acting' 'baking' 'basketball' 'birdwatching' 'blogging' 'board'
 'camping' 'coding' 'cooking' 'dancing' 'dinner' 'drawing' 'exercise'
 'exploring' 'fishing' 'games' 'gardening' 'gym' 'hiking' 'learning'
 'listening' 'lunch' 'meditation' 'meet' 'mentoring' 'movies' 'music'
 'painting' 'photography' 'play' 'playing' 'programming' 'reading' 'run'
 'singing' 'skiing' 'snowboarding' 'soccer' 'sports' 'studying' 'surfing'
 'swimming' 'teaching' 'tennis' 'travelling' 'video' 'volunteering' 'walk'
 'watching' 'writing' 'yoga']


KeyboardInterrupt: Interrupted by user

In [27]:
import re
import spacy
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load the spaCy English model with NER capabilities
nlp = spacy.load("en_core_web_sm")

# Load the dataset
activities_df = pd.read_csv("/content/activities.csv")
activities_corpus = activities_df["Activity"].tolist()

# Function to train an activity classifier
def train_activity_classifier(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    y = [1] * len(corpus)
    clf = MultinomialNB()
    clf.fit(X, y)
    return vectorizer, clf

# Train
activity_vectorizer, activity_clf = train_activity_classifier(activities_corpus)

def print_features(vectorizer):
    print("Features extracted by the vectorizer:")
    feature_names = vectorizer.get_feature_names_out()
    print(feature_names)

# Print features extracted by the vectorizer
print_features(activity_vectorizer)

# Function to extract date, time, and activity using NER and activity classification
def extract_info(user_input):
    doc = nlp(user_input)
    entities = {"DATE": None, "TIME": [], "ACTIVITY": []}

    # time extraction
    time_regex = r'\b(\d{1,2}:\d{2}\s*(?:am|pm))\b'
    time_matches = re.findall(time_regex, user_input)
    entities["TIME"] = time_matches

    for ent in doc.ents:
        if ent.label_ == "DATE":
            entities["DATE"] = ent.text

    # Classify the activity using the activity classifier
    X = activity_vectorizer.transform([user_input])
    predicted_label = activity_clf.predict(X)
    if predicted_label[0] == 1:
        activity_regex = r'(?:go|will go)\s+([\w\s]+?)\s*(?:and|$)'
        activities = re.findall(activity_regex, user_input, re.IGNORECASE)
        entities["ACTIVITY"] = activities

    print("Extracted information:", entities)
    return entities

def main():
    user_input = input("Please enter your schedule: ")
    # Extract information from user using NER and activity classification
    extracted_info = extract_info(user_input)

    # Generate JSON
    response = {}
    for i, time in enumerate(extracted_info["TIME"]):
        response[f"{time}-{extracted_info['DATE']}"] = extracted_info["ACTIVITY"][i]

    # Output
    print(json.dumps(response, indent=4))

if __name__ == "__main__":
    main()

Features extracted by the vectorizer:
['acting' 'baking' 'basketball' 'birdwatching' 'blogging' 'board'
 'camping' 'coding' 'cooking' 'dancing' 'dinner' 'drawing' 'exercise'
 'exploring' 'fishing' 'games' 'gardening' 'gym' 'hiking' 'learning'
 'listening' 'lunch' 'meditation' 'meet' 'mentoring' 'movies' 'music'
 'painting' 'photography' 'play' 'playing' 'programming' 'reading' 'run'
 'singing' 'skiing' 'snowboarding' 'soccer' 'sports' 'studying' 'surfing'
 'swimming' 'teaching' 'tennis' 'travelling' 'video' 'volunteering' 'walk'
 'watching' 'writing' 'yoga']
Please enter your schedule: On 31/03/2024 at 8:00 am I will go swimming and at 12:00 pm I will go coding and at 3:00 pm I will go to the gym
Extracted information: {'DATE': '31/03/2024', 'TIME': ['8:00 am', '12:00 pm', '3:00 pm'], 'ACTIVITY': ['swimming', 'coding', 'to the gym']}
{
    "8:00 am-31/03/2024": "swimming",
    "12:00 pm-31/03/2024": "coding",
    "3:00 pm-31/03/2024": "to the gym"
}


In [66]:
import re
import spacy
import pandas as pd
from dateutil import parser
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from dateutil.parser._parser import ParserError
import json

class ScheduleExtractor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.activity_vectorizer = CountVectorizer()
        self.activity_clf = None
        self.feature_names = None

    def train_activity_classifier(self, corpus):
        X = self.activity_vectorizer.fit_transform(corpus)
        y = [1] * len(corpus)
        clf = MultinomialNB()
        clf.fit(X, y)
        self.activity_clf = clf
        self.feature_names = self.activity_vectorizer.get_feature_names_out()
        print("Feature names (activities in the training dataset):")
        print(self.feature_names)

    def train(self, activities_file):
        activities_df = pd.read_csv(activities_file)
        activities_corpus = activities_df["Activity"].tolist()
        self.train_activity_classifier(activities_corpus)

    def preprocess_text(self, text):
        text = re.sub(r'[^\w\s]', '', text)
        return text

    def extract_info(self, user_input):
        user_input = self.preprocess_text(user_input)
        doc = self.nlp(user_input)
        entities = {"DATE": None, "TIME": [], "ACTIVITY": []}
        time_regex = r'\b(\d{1,2}:\d{2}\s*(?:am|pm))\b'
        time_matches = re.findall(time_regex, user_input)
        entities["TIME"] = time_matches
        for ent in doc.ents:
            if ent.label == "DATE":
                entities["DATE"] = ent.text
        X = self.activity_vectorizer.transform([user_input])
        predicted_label = self.activity_clf.predict(X)
        if predicted_label[0] == 1:
            activity_regex = r'\b((?:go\s+(?:to\s+)?|do|have)\s+\w+(?:\s+\w+)*)\b'
            activities = re.findall(activity_regex, user_input, re.IGNORECASE)
            entities["ACTIVITY"] = activities
        return entities

    def generate_json(self, extracted_info):
        response = {}
        date_obj = None
        if "tomorrow" in extracted_info["DATE"]:
            date_obj = (datetime.today() + timedelta(days=1)).date()
        elif "day after tomorrow" in extracted_info["DATE"]:
            date_obj = (datetime.today() + timedelta(days=2)).date()
        else:
            try:
                date_obj = parser.parse(extracted_info["DATE"], fuzzy=True).date()
            except (ValueError, ParserError):
                pass

        for activity, time in zip(extracted_info["ACTIVITY"], extracted_info["TIME"]):
            start_time = parser.parse(time, fuzzy=True).time()
            end_time = (parser.parse(time, fuzzy=True) + timedelta(hours=1)).time()
            original_activity = None
            for feature in self.feature_names:
                if activity in feature:
                    original_activity = feature
                    break
            if original_activity:
                if date_obj:
                    start_datetime = parser.parse(f"{date_obj} {start_time}")
                    end_datetime = parser.parse(f"{date_obj} {end_time}")
                    response[f"{start_datetime.strftime('%I:%M %p')}-{end_datetime.strftime('%I:%M %p')}"] = original_activity
                else:
                    response[f"{start_time.strftime('%I:%M %p')}-{end_time.strftime('%I:%M %p')}"] = original_activity
        if not response:
            print("No activities found in the input.")
            return
        return response

# Example usage
if __name__ == "__main__":
    schedule_extractor = ScheduleExtractor()
    schedule_extractor.train("activities.csv")

    user_input = input("Please enter your schedule: ")
    extracted_info = schedule_extractor.extract_info(user_input)
    json_output = schedule_extractor.generate_json(extracted_info)

    print(json.dumps(json_output, indent=4))

Feature names (activities in the training dataset):
['acting' 'baking' 'basketball' 'birdwatching' 'blogging' 'board'
 'camping' 'coding' 'cooking' 'dancing' 'dinner' 'drawing' 'exercise'
 'exploring' 'fishing' 'games' 'gardening' 'gym' 'hiking' 'learning'
 'listening' 'lunch' 'meditation' 'meet' 'mentoring' 'movies' 'music'
 'painting' 'photography' 'play' 'playing' 'programming' 'reading' 'run'
 'singing' 'skiing' 'snowboarding' 'soccer' 'sports' 'studying' 'surfing'
 'swimming' 'teaching' 'tennis' 'travelling' 'video' 'volunteering' 'walk'
 'watching' 'writing' 'yoga']
Please enter your schedule: On 31/03/2024 at 8:00 am I will go swimming and at 12:00 pm I will go coding and at 3:00 pm I will go to the gym


TypeError: argument of type 'NoneType' is not iterable